In [1]:
#Dependencies 
import pandas as pd 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config import gkey
import requests
import json
import time

In [2]:
data = "http://www.espn.com/nba/attendance/_/year/2019"
nba_attendance = pd.read_html(data)

In [3]:
nba_attendance=nba_attendance[0].drop([0])
nba_attendance.columns=nba_attendance.iloc[0]
nba_attendance=nba_attendance.drop([1])
nba_attendance=nba_attendance.drop(['GMS','AVG','PCT','RK'],axis=1)
nba_attendance.columns = ["team", "total_attendance"]
nba_attendance=nba_attendance.sort_values("team")
nba_attendance=nba_attendance[nba_attendance["total_attendance"]!="0"]
nba_attendance

,team,total_attendance
2,76ers,838092
18,Bucks,721692
3,Bulls,823475
9,Cavaliers,793337
12,Celtics,763584
21,Clippers,710327
27,Grizzlies,607564
28,Hawks,628440
6,Heat,805264
24,Hornets,676570


In [4]:
url = "https://en.wikipedia.org/wiki/List_of_National_Basketball_Association_arenas"
nba_capacities = pd.read_html(url)

In [5]:
nba_capacities = nba_capacities[0].drop(['Image','Location','Opened','Ref(s)'],axis=1)
nba_capacities["Lat"]=""
nba_capacities["Lng"]=""
nba_capacities

,Arena,Team(s),Capacity,Lat,Lng
0,American Airlines Arena,Miami Heat,19600,,
1,American Airlines Center,Dallas Mavericks,19200,,
2,Amway Center,Orlando Magic,18846,,
3,AT&T Center,San Antonio Spurs,18418,,
4,Bankers Life Fieldhouse,Indiana Pacers,17923,,
5,Barclays Center,Brooklyn Nets,17732,,
6,Capital One Arena,Washington Wizards,20356,,
7,Chase Center,Golden State Warriors,18064,,
8,Chesapeake Energy Arena,Oklahoma City Thunder,18203,,
9,FedExForum,Memphis Grizzlies,17794,,


In [6]:
params={"key":gkey}
for index,row in nba_capacities.iterrows():
    base_url="https://maps.googleapis.com/maps/api/geocode/json"
    arena=row["Arena"]
    params["address"]=arena
    result=requests.get(base_url, params=params)
    result=result.json()
    try:    
        nba_capacities.loc[index, "Lat"] = result["results"][0]["geometry"]["location"]["lat"]
        nba_capacities.loc[index, "Lng"] = result["results"][0]["geometry"]["location"]["lng"]
        time.sleep(1)
    except (IndexError, KeyError):
        print("results not found...skipping")
        time.sleep(1)
print("Complete")

Complete


In [16]:
nba_capacities=nba_capacities.sort_values("Team(s)")
nba_capacities.head()


,Arena,Team(s),Capacity,Lat,Lng
22,State Farm Arena,Atlanta Hawks,18118,33.7573,-84.3963
25,TD Garden,Boston Celtics,18624,42.3662,-71.0621
5,Barclays Center,Brooklyn Nets,17732,40.6826,-73.9754
19,Spectrum Center,Charlotte Hornets,19077,35.2252,-80.8394
27,United Center,Chicago Bulls,20917,41.8807,-87.6742


In [9]:
nba_attendance.head()

,team,total_attendance
2,76ers,838092
18,Bucks,721692
3,Bulls,823475
9,Cavaliers,793337
12,Celtics,763584


In [18]:
nba_data=pd.merge_asof(nba_capacities, nba_attendance, left_on="Team(s)",right_on="team")
nba_data

TypeError: No matching signature found